# Facebook Messenger Statistics

In [ ]:
from scripts import loader
from scripts import chatstat
import os
from plotly.subplots import make_subplots

# Load your chats
For more instructions go [here](https://github.com/simonwongwong/Facebook-Messenger-Statistics#how-to-use)

When you extract your data from Facebook it should contain a folder called `messages`, and there should be an `inbox` folder within the `messages` folder.
Pass in the path of the `inbox` folder into `parse_from_json`

In [ ]:
cs = chatstat.ChatStat(*loader.parse_from_json("path/to/your_facebook_activity/messages/inbox"))

# All-time stats
Here are some statistics using all available chat data

## Who are you chatting with?

In [ ]:
who = make_subplots(rows=2, cols=2, specs=[[{"type": "bar"}, {"type": "bar"}], [{"type": "bar", "colspan": 2}, None]],
                    subplot_titles=['# of Messages by Sender', '# of Chats by Sender','# of Messages by Chat'])
who.add_trace(cs.sent_from(top=10,omit_first=True,kind='bar',show=False), row=1,col=1)
who.add_trace(cs.chat_counts(top=10,omit_first=True,show=False), row=1,col=2)
who.add_trace(cs.biggest_chat(top=10,kind='bar',show=False), row=2,col=1)
who.update_layout(height=950, width=950, showlegend=False, title_text="Stats by Number of Messages")
who.show()

who_pct = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
who_pct.add_trace(cs.sent_from(top=10,omit_first=True,kind='pie',show=False), row=1,col=1)
who_pct.add_trace(cs.biggest_chat(top=10,kind='pie',show=False), row=1,col=2)
who_pct.update_layout(height=475, width=950, showlegend=True, title_text="Stats by Proportion of Messages")



## How are you chatting them?

In [ ]:
how = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
how.add_trace(cs.msg_types(show=False), row=1,col=1)
how.add_trace(cs.chat_types(show=False), row=1,col=2)

## When are you the most chatty?
i.e. when are you procrastinating

In [ ]:
cs.time_stats(show=True)

## What words are you using?

In [ ]:
cs.word_counts(length=[3,5])

# View stats by sender
set `sender_name` to your own name to see stats based on all messages sent by you

set `sender_name` to one of your friend's names to see stats based on all messages sent by them

In [ ]:
sender_name = "name_of_account"
cs.personal_stats(sender_name)

# View stats per chat

set `chat_name` to the name/title of any chat or group chat to see stats about messages sent to that chat from all participants

How balanced are your chats?

In [ ]:
chat_name = "name_of_account"
cs.stat_by_chat(chat_name)


In [ ]:
# For report in HTML format From the Repository owner suggestion

# you have to choose the folder with path /your_facebook_activity/messages/inbox choose inbox
!python ./scripts/generate_report.py

### ----------------------------------------------------------------
### ----------------------------------------------------------------
### ----------------------------------------------------------------
### ----------------------------------------------------------------

### Other Functions

Reacting for messages counter

In [ ]:
from datetime import datetime
from plotly.subplots import make_subplots
import pandas as pd
import plotly.graph_objects as go
from functools import wraps
import json

def count_reactions(messages):
    stats = {}
    total_reactions = {}
    total_reactions_by_sender = {}
    for message in messages:
        sender = message['sender_name']
        if 'reactions' in message:
            reactions = message['reactions']
            if sender not in stats:
                stats[sender] = {}
            for reaction in reactions:
                # Decode the reaction as UTF-8
                reaction_type = bytes(reaction['reaction'], 'latin-1').decode('utf-8')
                if reaction_type not in stats[sender]:
                    stats[sender][reaction_type] = 0
                stats[sender][reaction_type] += 1
                total_reactions[reaction_type] = total_reactions.get(reaction_type, 0) + 1
                total_reactions_by_sender[sender] = total_reactions_by_sender.get(sender, 0) + 1



    
    # Generate pie chart for total reactions by sender
    total_reactions_by_sender = dict(sorted(total_reactions_by_sender.items(), key=lambda item: item[1], reverse=True))

    # Group "Other" reactions
    total_count = sum(total_reactions_by_sender.values())
    other_count = sum(count for sender, count in total_reactions_by_sender.items() if count / total_count < 0.005)
    total_reactions_by_sender = {sender: count for sender, count in total_reactions_by_sender.items() if count / total_count >= 0.005}
    if other_count > 0:
        total_reactions_by_sender['Others'] = other_count

    # Add count to labels
    labels = [f'{sender}\n({count})' for sender, count in total_reactions_by_sender.items()]

    fig = go.Figure(data=[go.Pie(labels=labels, values=list(total_reactions_by_sender.values()))])
    fig.update_layout(title_text='Total reactions by sender')
    fig.show()





    # Sort reactions by total count
    sorted_reactions = sorted(total_reactions.items(), key=lambda item: item[1], reverse=True)

    # Generate pie chart for each reaction
    for reaction, _ in sorted_reactions:
        senders = {sender: reactions[reaction] for sender, reactions in stats.items() if reaction in reactions}
        # Sort senders from most to least
        senders = dict(sorted(senders.items(), key=lambda item: item[1], reverse=True))

        # Group "Other" senders
        other_count = sum(count for sender, count in senders.items() if count / total_reactions[reaction] < 0.005)
        senders = {sender: count for sender, count in senders.items() if count / total_reactions[reaction] >= 0.005}
        if other_count > 0:
            senders['Others'] = other_count

        fig = go.Figure(data=[go.Pie(labels=list(senders.keys()), values=list(senders.values()))])
        fig.update_layout(title_text=f'Senders for {reaction} reaction')
        fig.show()

    return stats

with open('path/to/your/file/of/the/conversation.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    messages = data['messages']

count_reactions(messages)

In [ ]:
def count_reactions_by_sender(messages, sender_name):
     stats = {}
     total_reactions = {}
     total_reactions_by_sender = {}
     for message in messages:
          sender = message['sender_name']
          if 'reactions' in message:
               reactions = message['reactions']
               if sender == sender_name:
                    for reaction in reactions:
                         reacter = reaction['actor']
                         # Decode the reaction as UTF-8
                         reaction_type = bytes(reaction['reaction'], 'latin-1').decode('utf-8')
                         if reacter not in stats:
                              stats[reacter] = {}
                         if reaction_type not in stats[reacter]:
                              stats[reacter][reaction_type] = 0
                         stats[reacter][reaction_type] += 1
                         total_reactions[reaction_type] = total_reactions.get(reaction_type, 0) + 1
                         total_reactions_by_sender[reacter] = total_reactions_by_sender.get(reacter, 0) + 1

     # Sort reactions by total count
     sorted_reactions = sorted(total_reactions_by_sender.items(), key=lambda item: item[1], reverse=True)

     # Group "Other" senders
     other_count = sum(count for sender, count in sorted_reactions[20:])
     top_reactions_by_sender = dict(sorted_reactions[:20])
     if other_count > 0:
          top_reactions_by_sender['Others'] = other_count

     # Generate pie chart
     labels = list(top_reactions_by_sender.keys())
     values = list(top_reactions_by_sender.values())

     fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
     fig.update_layout(title_text=f'Top 10 Reactions to {sender_name}')
     fig.show()

     return stats

# Example usage
count_reactions_by_sender(messages, "name_of_account")



In [ ]:
def count_reactions_by_reacter(messages, reacter_name):
     stats = {}
     total_reactions = {}
     total_reactions_to_recipients = {}
     
     for message in messages:
          if 'reactions' in message:
               reactions = message['reactions']
               for reaction in reactions:
                    reacter = reaction['actor']
                    # Decode the reaction as UTF-8
                    reaction_type = bytes(reaction['reaction'], 'latin-1').decode('utf-8')
                    
                    if reacter == reacter_name:
                         recipient = message['sender_name']
                         
                         if recipient not in stats:
                              stats[recipient] = {}
                         
                         if reaction_type not in stats[recipient]:
                              stats[recipient][reaction_type] = 0
                         
                         stats[recipient][reaction_type] += 1
                         
                         total_reactions[reaction_type] = total_reactions.get(reaction_type, 0) + 1
                         
                         if recipient not in total_reactions_to_recipients:
                              total_reactions_to_recipients[recipient] = 0
                         
                         total_reactions_to_recipients[recipient] += 1
     
     # Sort recipients by total count
     most_reacted_recipient = max(total_reactions_to_recipients, key=total_reactions_to_recipients.get, default=None)
     
     # Generate pie chart for total reactions to recipients
     labels = list(total_reactions_to_recipients.keys())
     values = list(total_reactions_to_recipients.values())
     
     fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
     fig.update_layout(title_text=f'Top 10 Reactions by {reacter_name}')
     fig.show()
     
     # Get total reactions by the reacter
     total_reactions_by_reacter = sum(total_reactions_to_recipients.values())
     
     return total_reactions_by_reacter, most_reacted_recipient

# Example usage
account="name_of_account"
total_reactions_by_reacter, most_reacted_recipient = count_reactions_by_reacter(messages, account)
print(f"Total reactions by {account}: {total_reactions_by_reacter}")
print(f"the most Person reacted to {account} is: {most_reacted_recipient}")
